<img src="https://full-stack-assets.s3.eu-west-3.amazonaws.com/M08-deep-learning/AT%26T_logo_2016.svg" alt="AT&T LOGO" width="50%" />

# AT&T SPAM detector

## Présentation de l’entreprise 📇

AT&T Inc. est une entreprise multinationale américaine de télécommunications, dont le siège est situé à la Whitacre Tower, dans le centre-ville de Dallas, au Texas.  
C’est la plus grande entreprise de télécommunications au monde en termes de chiffre d’affaires et le troisième plus grand fournisseur de services de téléphonie mobile aux États-Unis.  

En 2022, AT&T était classée **13ᵉ** au classement Fortune 500 des plus grandes entreprises américaines, avec un chiffre d’affaires de **168,8 milliards de dollars** 😮

---

## Projet 🚧

L’un des principaux points de douleur rencontrés par les utilisateurs d’AT&T est leur exposition constante aux **messages SPAM**.

AT&T a, pendant un temps, été capable de signaler manuellement les messages indésirables, mais l’entreprise recherche désormais une **solution automatisée** permettant de détecter les spams afin de mieux protéger ses utilisateurs.

---

## Objectifs 🎯

Ton objectif est de construire un **détecteur de spam** capable de signaler automatiquement les messages indésirables au moment où ils arrivent, en se basant **uniquement sur le contenu des SMS**.

---


## Périmètre du projet 🖼️

Pour commencer, AT&T souhaite que tu utilises le jeu de données suivant :

[Télécharger le jeu de données](https://full-stack-bigdata-datasets.s3.eu-west-3.amazonaws.com/Deep+Learning/project/spam.csv)

---


In [ ]:
#%cd Data
#%pwd
#!wget https://full-stack-bigdata-datasets.s3.eu-west-3.amazonaws.com/Deep+Learning/project/spam.csv
#%cd ..


## Aides 🦮

Pour t’aider à mener ce projet à bien, voici quelques conseils utiles :

### Commencer simplement

Un bon modèle de deep learning n’a pas nécessairement besoin d’être extrêmement complexe !

### Transfer learning

Tu n’as pas accès à une très grande quantité de données. Exploiter la puissance d’un modèle plus sophistiqué, entraîné sur des milliards d’observations, pourrait donc être une bonne approche.

---

## Livrables 📬

Pour valider ce projet, ton équipe devra :

- Rédiger un notebook qui effectue le prétraitement des données et entraîne un ou plusieurs modèles de deep learning afin de prédire si un SMS est un spam ou non (ham)
- Présenter clairement les performances obtenues

&nbsp;

---



## Processus envisagé pour le projet : AT&T

Pour un projet comme AT&T, le déroulement pourrait être le suivant :

- [x] Trouver un bon modèle LLM capable de gérer la détection de spam  
- [x] L’entraîner et l’améliorer à l’aide des données disponibles  
- [x] Tester et comparer les performances avec les meilleurs modèles disponibles sur Hugging Face  

---

## Objectifs concrets

### Quels outils dois-tu utiliser ?

- Lightning AI et Google Colab pour l’entraînement et le fine-tuning des modèles

### Quels processus dois-tu mettre en place ?

- Sélection d’un modèle pré-entraîné  
- Fine-tuning du ou des modèles  
- Évaluation des performances  

### À quelles questions dois-tu répondre ?

- Spam ou ham ? Telle est la question.

### Quels problèmes dois-tu résoudre ?

- Nettoyage et préparation des données  
- Fine-tuning du modèle  

### Quels fichiers spécifiques dois-tu rendre pour la certification ?

- Un notebook  
- Un court rapport présentant les résultats de l’évaluation des modèles


In [ ]:
# if use of google colab
#from google.colab import drive
#drive.mount('/content/drive')

# Rendu


In [ ]:
pip install datasets transformers evaluate rouge_score -q

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification
from transformers import pipeline,TrainingArguments,Trainer,EvalPrediction

In [ ]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

In [ ]:
#ROOT_DIR="/content/drive/MyDrive/Colab Notebooks/Jedha/dsfs-ft-35/att"
ROOT_DIR="."

Toilettage du fichier pour retirer les caractères invalides.

In [ ]:
import unicodedata

def clean_international_text(text):
    # Normalize Unicode characters
    normalized = unicodedata.normalize('NFKD', text)
    # Remove non-ASCII characters
    ascii_text = normalized.encode('ASCII', 'ignore').decode('ASCII')
    return ascii_text

In [ ]:
filenameIn=ROOT_DIR+"/Data/spam.csv"
output=clean_international_text(open(filenameIn, "r",encoding="ISO-8859-1").read())
filename=ROOT_DIR+"/Data/spam_clean.csv"
with open(filename, "w") as f:
    f.write(output)

In [ ]:
spam_dataset = pd.read_csv(ROOT_DIR+"/Data/spam_clean.csv")#,encoding="ISO-8859-1"

In [ ]:
spam_dataset.shape

## Dataset exploration

Nous explorons le jeu de données pour comprendre sa structure par rapport au problème d'analyse de spam qui nous préoccupe.

In [ ]:
display(spam_dataset.head())
display(spam_dataset.info())
display(spam_dataset.describe())
display(spam_dataset.shape)


# ETL sur le jeu de données de SPAM

In [ ]:
#spam_dataset.dropna(inplace=True)
spam_dataset.rename(columns={ 'v1': 'type','v2': 'line1', 'Unnamed: 2': 'line2','Unnamed: 3': 'line3','Unnamed: 4': 'line4'}, inplace=True)
spam_dataset.shape

Nous fusionnons les lignes du texto en un unique message.

In [ ]:
spam_dataset['message'] = spam_dataset[['line1', 'line2', 'line3', 'line4']].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
spam_dataset['message'] =spam_dataset['message'].apply(lambda x: x.replace('\\', ''))
#spam_dataset['message'] = spam_dataset['line1']

Nous transformons la colonne 'v1' en une colonne binaire 'spam' où 'spam' devient 1 et 'ham' devient 0.

In [ ]:
mapping = {
  "spam": 1,
  "ham": 0
}
spam_dataset["spam"] = spam_dataset["type"].map(mapping)
spam_dataset = spam_dataset[['spam', 'message','type']]
spam_dataset.shape

In [ ]:
spam_new=spam_dataset.rename(columns={ 'message': 'text','spam': 'label', 'type': 'label_text'})

In [ ]:
spam_new.shape

## Verification de l'intégration

In [ ]:
print(spam_dataset.isnull().sum())
spam_dataset

In [ ]:
from datasets import Dataset

spam_dataset = Dataset.from_pandas(spam_new) #spam_dataset

## Preprocessing du dataset
Il est nécessaire de convertir le texte en suite de nombres avec un "tokenizer'.

In [ ]:
#checkpoint = "bert-base-uncased"
#checkpoint = "AventIQ-AI/bert-spam-detection"

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding=True)

tokenized_datasets = spam_dataset.map(tokenize_function, batched=True)
samples = tokenized_datasets[:2]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}
[len(x) for x in samples["input_ids"]]


In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
pip install 'accelerate>=0.26.0'

## Jeux d'entrainement et de test
Le jeu de données est propre et prêt, il faut maintenant le diviser en un jeu d'entrainement, un jeu de validation et un jeu de test.

In [ ]:
tokenized_datasets=tokenized_datasets.train_test_split(test_size=0.2)


In [ ]:
tokenized_datasets

In [ ]:
training_args = TrainingArguments("test-trainer", report_to='none')

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

In [ ]:
from transformers import Trainer

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    processing_class=tokenizer,
)

In [ ]:
trainer.train()

In [ ]:
#tokenized_datasets = tokenized_datasets.rename_column("spam", "label")
display(tokenized_datasets)


### Evaluation

In [ ]:


predictions = trainer.predict(tokenized_datasets["test"])
print(predictions.predictions.shape, predictions.label_ids.shape)

In [ ]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

predictions

In [ ]:
from sklearn.metrics import classification_report

true_labels = tokenized_datasets['test']['label']
print(classification_report(true_labels, preds))

In [ ]:
%pip install plotly 

In [ ]:
confusion_matrix(true_labels, preds)

In [ ]:
from sklearn.metrics import confusion_matrix
import plotly.express as px

cm = pd.DataFrame(confusion_matrix(true_labels, preds),
                  index=['ham', 'spam'],
                  columns=['ham', 'spam'])
px.imshow(cm, text_auto=True)

In [ ]:
%pip install evaluate
%pip install seqeval

In [ ]:
import evaluate

seqeval = evaluate.load('seqeval')


def compute_metrics(p: EvalPrediction):
    return seqeval.compute(predictions=p.predictions, references=p.label_ids)

In [ ]:
training_args = TrainingArguments("test-trainer", eval_strategy="epoch") # push_to_hub=True if you wish to push your model to the hugging face hub
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

## Question pipeline

In [ ]:
spam_sample = spam_new.sample(3000)
display(spam_sample.head())

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
classifier = pipeline("question-answering", model="distilbert/distilbert-base-uncased-finetuned-sst-2-english")
#classifier = pipeline("text-classification", model="distilbert/distilbert-base-cased-distilled-squad",
#                      tokenizer="google-bert/bert-base-cased")
question = "The context is a sms. Answer with one word betwween ham and spam. Is it a spam or a ham ?"
prep=[{"context": ele, "question": question } for ele in spam_sample['text'].to_list()]
spam_preds = classifier(prep)
#map={'POSITIVE' :'ham', 'NEGATIVE': 'spam'}
#spam_preds = [{'label': map[ele['label']],'score':ele['score']} for ele in spam_preds]
#spam_preds['label'].map(map)
display(spam_preds[:10])

## Nouveau modèle

In [ ]:
spam_sample = spam_new.sample(3000)
display(spam_sample.head())

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
classifier = pipeline("text-classification", model="distilbert/distilbert-base-uncased-finetuned-sst-2-english")
#classifier = pipeline("text-classification", model="distilbert/distilbert-base-cased-distilled-squad",
#                      tokenizer="google-bert/bert-base-cased")
context = "There are three major types of rock: igneous, sedimentary, and metamorphic. The rock cycle is an important concept in geology which illustrates the relationships between these three types of rock, and magma. When a rock crystallizes from melt (magma and/or lava), it is an igneous rock. This rock can be weathered and eroded, and then redeposited and lithified into a sedimentary rock, or be turned into a metamorphic rock due to heat and pressure that change the mineral content of the rock which gives it a characteristic fabric. The sedimentary rock can then be subsequently turned into a metamorphic rock due to heat and pressure and is then weathered, eroded, deposited, and lithified, ultimately becoming a sedimentary rock. Sedimentary rock may also be re-eroded and redeposited, and metamorphic rock may also undergo additional metamorphism. All three types of rocks may be re-melted; when this happens, a new magma is formed, from which an igneous rock may once again crystallize."
question = "What are the three major types of rock?"
qa_pipe = pipeline("question-answering")
qa_pipe({ "context": context, "question": question })
spam_preds = classifier(spam_sample['text'].to_list())
map={'POSITIVE' :'ham', 'NEGATIVE': 'spam'}
spam_preds = [{'label': map[ele['label']],'score':ele['score']} for ele in spam_preds]
#spam_preds['label'].map(map)
display(spam_preds[:10])

In [ ]:
spam_preds = pd.DataFrame(spam_preds)
spam_preds.head()
spam_preds.shape

In [ ]:
from sklearn.metrics import classification_report

true_labels = spam_sample['label_text']
pred_labels = spam_preds['label']

print(classification_report(true_labels, pred_labels))

In [ ]:
from sklearn.metrics import confusion_matrix
import plotly.express as px

cm = pd.DataFrame(confusion_matrix(true_labels, pred_labels),
                  index=['ham',  'spam'],
                  columns=['ham',  'spam'])

px.imshow(cm, text_auto=True)

## Second model

In [ ]:
from datasets import load_dataset

In [ ]:

spam_new.head(30)

In [ ]:
spam_new.shape

In [ ]:
spam_sample = spam_new.sample(3000)
display(spam_sample.head())

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
#classifier = pipeline("text-classification", model="distilbert/distilbert-base-uncased-finetuned-sst-2-english") Non
model_name = "AventIQ-AI/bert-spam-detection"
tokenizer = BertTokenizer.from_pretrained(model_name)
googletokenizer="google-bert/bert-base-cased"
model = BertForSequenceClassification.from_pretrained(model_name)
classifier = pipeline("text-classification", model=model_name,tokenizer=tokenizer)
#classifier = pipeline("text-classification", model="distilbert/distilbert-base-cased-distilled-squad",
#                      tokenizer="google-bert/bert-base-cased")
spam_preds = classifier(spam_sample['text'].to_list())
map={'LABEL_0': 'ham', 'LABEL_1': 'spam'}
spam_preds = [{'label': map[ele['label']],'score':ele['score']} for ele in spam_preds]
#spam_preds['label'].map(map)
display(spam_preds[:10])

In [ ]:
spam_preds = pd.DataFrame(spam_preds)
spam_preds.head()
spam_preds.shape

In [ ]:
spam_preds
spam_sample


In [ ]:
from sklearn.metrics import classification_report

true_labels = spam_sample['label_text']
pred_labels = spam_preds['label']

print(classification_report(true_labels, pred_labels))

In [ ]:
confusion_matrix(true_labels, pred_labels)

In [ ]:
from sklearn.metrics import confusion_matrix
import plotly.express as px

cm = pd.DataFrame(confusion_matrix(true_labels, pred_labels),
                  index=['ham',  'spam'],
                  columns=['ham',  'spam'])

px.imshow(cm, text_auto=True)

##Troisième modèle


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
#classifier = pipeline("text-classification", model="distilbert/distilbert-base-uncased-finetuned-sst-2-english") Non
classifier = pipeline("text-classification", model="mrm8488/bert-tiny-finetuned-sms-spam-detection")
spam_preds = classifier(spam_sample['text'].to_list())
map={'LABEL_0': 'ham', 'LABEL_1': 'spam'}
spam_preds = [{'label': map[ele['label']],'score':ele['score']} for ele in spam_preds]
#spam_preds['label'].map(map)
display(spam_preds[:10])

In [ ]:
spam_preds = pd.DataFrame(spam_preds)
spam_preds.head()
spam_preds.shape

In [ ]:
from sklearn.metrics import classification_report

true_labels = spam_sample['label_text']
pred_labels = spam_preds['label']

print(classification_report(true_labels, pred_labels))

In [ ]:
from sklearn.metrics import confusion_matrix
import plotly.express as px

cm = pd.DataFrame(confusion_matrix(true_labels, pred_labels),
                  index=['ham',  'spam'],
                  columns=['ham',  'spam'])

px.imshow(cm, text_auto=True)